In [ ]:
import os

import optuna
from optuna.trial import TrialState
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
import torchvision
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

DEVICE = torch.device("cpu")
BATCHSIZE = 32
CLASSES = 4
DIR = os.getcwd()
EPOCHS = 20
N_TRAIN_EXAMPLES = BATCHSIZE * 15
N_VALID_EXAMPLES = BATCHSIZE * 5


def define_model(trial):
    # We optimize the number of layers, hidden units and dropout ratio in each layer.
    n_layers = trial.suggest_int("n_layers", 1, 4)
    layers = []
    in_channels = [3, 8, 16, 32]
    out_channels = [8, 16, 32, 64]
    dimension = [1, 8*111*111, 16*54*54, 32*26*26, 64*12*12]
    for i in range(1, n_layers):
        layers.append(nn.Conv2d(in_channels=in_channels[i-1], out_channels=out_channels[i-1], kernel_size=3))
        layers.append(nn.BatchNorm2d(out_channels[i-1]))
        layers.append(nn.ReLU())
        layers.append(nn.MaxPool2d(kernel_size=2))
    layers.append(nn.Flatten())
    layers.append(nn.Linear(dimension[n_layers - 1], CLASSES))
    layers.append(nn.LogSoftmax(dim=1))
    return nn.Sequential(*layers)


def get_data():
    train_path = ('data_folder/Training/')
    test_path = ('data_folder/Testing/')
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    train_dataset = ImageFolder(root=train_path, transform=transform)
    test_dataset = ImageFolder(root=test_path, transform=transform)
    train_loader=DataLoader(
        train_dataset,
        batch_size=32, shuffle=True,
    )
    test_loader=DataLoader(
        test_dataset,
        batch_size=32, shuffle=True,
    )
    return train_loader, test_loader


def objective(trial):
    # Generate the model.
    model = define_model(trial).to(DEVICE)
    # Generate the optimizers.
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    # Get the FashionMNIST dataset.
    train_dataloader, val_dataloader = get_data()
    train_loss_log, train_acc_log, val_loss_log, val_acc_log = [], [], [], []
    # Training of the model.
    for epoch in range(EPOCHS):
        # тренировка
        train_epoch_loss, train_epoch_true_hits = torch.empty(0), torch.empty(0)
        model.train()
        for i, (imgs, labels) in enumerate(train_dataloader):
            imgs, labels = imgs.to(device), labels.to(device)

            y_pred = model(imgs)
            loss = criterion(y_pred, labels)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            # log loss for the current epoch and the whole training history
            train_epoch_loss = torch.cat(
                (train_epoch_loss, loss.cpu().unsqueeze(0) / labels.cpu().size(0))
            )
            train_loss_log.append(loss.cpu().data / labels.cpu().size(0))

            # log accuracy for the current epoch and the whole training history
            pred_classes = torch.argmax(y_pred.cpu(), dim=-1)
            train_epoch_true_hits = torch.cat(
                (
                    train_epoch_true_hits,
                    (pred_classes == labels.cpu()).sum().unsqueeze(0),
                )
            )
            train_acc_log.append(
                (pred_classes == labels.cpu()).sum() / labels.cpu().shape[0]
            )

        # валидация
        val_epoch_loss, val_epoch_true_hits = torch.empty(0), torch.empty(0)
        model.eval()
        with torch.no_grad():
            for i, (imgs, labels) in enumerate(val_dataloader):
                imgs, labels = imgs.to(device), labels.to(device)

                y_pred = model(imgs)
                loss = criterion(y_pred, labels)
                val_epoch_loss = torch.cat(
                    (val_epoch_loss, loss.cpu().unsqueeze(0) / labels.cpu().size(0))
                )

                pred_classes = torch.argmax(y_pred.cpu(), dim=-1)
                val_epoch_true_hits = torch.cat(
                    (
                        val_epoch_true_hits,
                        (pred_classes == labels.cpu()).sum().unsqueeze(0),
                    )
                )

        val_loss_log.append(val_epoch_loss.mean())
        val_acc_log.append(
            val_epoch_true_hits.sum()
            / val_epoch_true_hits.size(0)
            / val_dataloader.batch_size
        )
        accuracy = (val_epoch_true_hits.sum()
                / val_epoch_true_hits.size(0)
                / val_dataloader.batch_size).item()


        trial.report(accuracy, epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return accuracy


if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=75, timeout=6000)

    pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
    complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2024-01-07 16:26:33,314] A new study created in memory with name: no-name-96c11516-1f02-4da6-8186-3e1c0ad92236
[I 2024-01-07 16:37:25,383] Trial 0 finished with value: 0.692307710647583 and parameters: {'n_layers': 2, 'optimizer': 'RMSprop', 'lr': 0.009197209479604193}. Best is trial 0 with value: 0.692307710647583.


Study statistics: 
  Number of finished trials:  1
  Number of pruned trials:  0
  Number of complete trials:  1
Best trial:
  Value:  0.692307710647583
  Params: 
    n_layers: 2
    optimizer: RMSprop
    lr: 0.009197209479604193


In [ ]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"

In [ ]:
device = "cpu"

In [ ]:
import os

import optuna
from optuna.trial import TrialState
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
import torchvision
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

DEVICE = torch.device("cpu")
BATCHSIZE = 32
CLASSES = 4
DIR = os.getcwd()
EPOCHS = 10
N_TRAIN_EXAMPLES = BATCHSIZE * 45
N_VALID_EXAMPLES = BATCHSIZE * 7


def define_model(trial):
    # We optimize the number of layers, hidden units and dropout ratio in each layer.
    n_layers = trial.suggest_int("n_layers", 1, 4)
    layers = []
    in_channels = [3, 8, 16, 32]
    out_channels = [8, 16, 32, 64]
    dimension = [1, 8*111*111, 16*54*54, 32*26*26, 64*12*12]
    for i in range(n_layers):
        layers.append(nn.Conv2d(in_channels=in_channels[i], out_channels=out_channels[i], kernel_size=3))
        layers.append(nn.BatchNorm2d(out_channels[i]))
        layers.append(nn.ReLU())
        layers.append(nn.MaxPool2d(kernel_size=2))
    layers.append(nn.Flatten())
    layers.append(nn.Linear(dimension[n_layers], CLASSES))
    layers.append(nn.LogSoftmax(dim=1))
    return nn.Sequential(*layers)


def get_data():
    train_path = ('data_folder/Training/')
    test_path = ('data_folder/Testing/')
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    train_dataset = ImageFolder(root=train_path, transform=transform)
    test_dataset = ImageFolder(root=test_path, transform=transform)
    train_loader=DataLoader(
        train_dataset,
        batch_size=32, shuffle=True,
    )
    test_loader=DataLoader(
        test_dataset,
        batch_size=32, shuffle=True,
    )
    return train_loader, test_loader


def objective(trial):
    # Generate the model.
    model = define_model(trial).to(DEVICE)
    # Generate the optimizers.
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    # Get the FashionMNIST dataset.
    train_dataloader, val_dataloader = get_data()
    train_loss_log, train_acc_log, val_loss_log, val_acc_log = [], [], [], []
    # Training of the model.
    for epoch in range(EPOCHS):
        # тренировка
        train_epoch_loss, train_epoch_true_hits = torch.empty(0), torch.empty(0)
        model.train()
        for i, (imgs, labels) in enumerate(train_dataloader):
            # Limiting training data for faster epochs.
            if i * BATCHSIZE >= N_TRAIN_EXAMPLES:
                break
            imgs, labels = imgs.to(device), labels.to(device)

            y_pred = model(imgs)
            loss = criterion(y_pred, labels)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            # log loss for the current epoch and the whole training history
            train_epoch_loss = torch.cat(
                (train_epoch_loss, loss.cpu().unsqueeze(0) / labels.cpu().size(0))
            )
            train_loss_log.append(loss.cpu().data / labels.cpu().size(0))

            # log accuracy for the current epoch and the whole training history
            pred_classes = torch.argmax(y_pred.cpu(), dim=-1)
            train_epoch_true_hits = torch.cat(
                (
                    train_epoch_true_hits,
                    (pred_classes == labels.cpu()).sum().unsqueeze(0),
                )
            )
            train_acc_log.append(
                (pred_classes == labels.cpu()).sum() / labels.cpu().shape[0]
            )

        # валидация
        val_epoch_loss, val_epoch_true_hits = torch.empty(0), torch.empty(0)
        model.eval()
        with torch.no_grad():
            for i, (imgs, labels) in enumerate(val_dataloader):
                if i * BATCHSIZE >= N_VALID_EXAMPLES:
                    break
                imgs, labels = imgs.to(device), labels.to(device)

                y_pred = model(imgs)
                loss = criterion(y_pred, labels)
                val_epoch_loss = torch.cat(
                    (val_epoch_loss, loss.cpu().unsqueeze(0) / labels.cpu().size(0))
                )

                pred_classes = torch.argmax(y_pred.cpu(), dim=-1)
                val_epoch_true_hits = torch.cat(
                    (
                        val_epoch_true_hits,
                        (pred_classes == labels.cpu()).sum().unsqueeze(0),
                    )
                )

        val_loss_log.append(val_epoch_loss.mean())
        val_acc_log.append(
            val_epoch_true_hits.sum()
            / val_epoch_true_hits.size(0)
            / val_dataloader.batch_size
        )
        accuracy = (val_epoch_true_hits.sum()
                / val_epoch_true_hits.size(0)
                / val_dataloader.batch_size).item()


        trial.report(accuracy, epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return accuracy


if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=75, timeout=900)

    pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
    complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2024-01-08 17:00:26,645] A new study created in memory with name: no-name-c0a30014-15fb-4efc-bfcc-2107aef4e53b
[I 2024-01-08 17:05:14,966] Trial 0 finished with value: 0.7946428656578064 and parameters: {'n_layers': 3, 'optimizer': 'Adam', 'lr': 0.0007915009678028183}. Best is trial 0 with value: 0.7946428656578064.
[I 2024-01-08 17:08:46,201] Trial 1 finished with value: 0.6696428656578064 and parameters: {'n_layers': 1, 'optimizer': 'RMSprop', 'lr': 4.3625382758855986e-05}. Best is trial 0 with value: 0.7946428656578064.
[I 2024-01-08 17:12:24,076] Trial 2 finished with value: 0.5848214030265808 and parameters: {'n_layers': 3, 'optimizer': 'RMSprop', 'lr': 0.014219339653230799}. Best is trial 0 with value: 0.7946428656578064.
[I 2024-01-08 17:15:52,311] Trial 3 finished with value: 0.4642857015132904 and parameters: {'n_layers': 2, 'optimizer': 'SGD', 'lr': 5.857129450914013e-05}. Best is trial 0 with value: 0.7946428656578064.


Study statistics: 
  Number of finished trials:  4
  Number of pruned trials:  0
  Number of complete trials:  4
Best trial:
  Value:  0.7946428656578064
  Params: 
    n_layers: 3
    optimizer: Adam
    lr: 0.0007915009678028183


In [ ]:
print(1)

1


In [ ]:
import os

import optuna
from optuna.trial import TrialState
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
import torchvision
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

DEVICE = torch.device("cpu")
BATCHSIZE = 32
CLASSES = 4
DIR = os.getcwd()
EPOCHS = 10
N_TRAIN_EXAMPLES = BATCHSIZE * 45
N_VALID_EXAMPLES = BATCHSIZE * 11


def define_model(trial):
    # We optimize the number of layers, hidden units and dropout ratio in each layer.
    n_layers = trial.suggest_int("n_layers", 1, 4)
    layers = []
    in_channels = [3, 8, 16, 32]
    out_channels = [8, 16, 32, 64]
    dimension = [1, 8*111*111, 16*54*54, 32*26*26, 64*12*12]
    for i in range(n_layers):
        layers.append(nn.Conv2d(in_channels=in_channels[i], out_channels=out_channels[i], kernel_size=3))
        layers.append(nn.BatchNorm2d(out_channels[i]))
        layers.append(nn.ReLU())
        layers.append(nn.MaxPool2d(kernel_size=2))
    layers.append(nn.Flatten())
    layers.append(nn.Linear(dimension[n_layers], CLASSES))
    layers.append(nn.LogSoftmax(dim=1))
    return nn.Sequential(*layers)


def get_data():
    train_path = ('data_folder/Training/')
    test_path = ('data_folder/Testing/')
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    train_dataset = ImageFolder(root=train_path, transform=transform)
    test_dataset = ImageFolder(root=test_path, transform=transform)
    train_loader=DataLoader(
        train_dataset,
        batch_size=32, shuffle=True,
    )
    test_loader=DataLoader(
        test_dataset,
        batch_size=32, shuffle=True,
    )
    return train_loader, test_loader


def objective(trial):
    # Generate the model.
    model = define_model(trial).to(DEVICE)
    # Generate the optimizers.
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    # Get the FashionMNIST dataset.
    train_dataloader, val_dataloader = get_data()
    train_loss_log, train_acc_log, val_loss_log, val_acc_log = [], [], [], []
    # Training of the model.
    for epoch in range(EPOCHS):
        # тренировка
        train_epoch_loss, train_epoch_true_hits = torch.empty(0), torch.empty(0)
        model.train()
        for i, (imgs, labels) in enumerate(train_dataloader):
            # Limiting training data for faster epochs.
            if i * BATCHSIZE >= N_TRAIN_EXAMPLES:
                break
            imgs, labels = imgs.to(device), labels.to(device)

            y_pred = model(imgs)
            loss = criterion(y_pred, labels)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            # log loss for the current epoch and the whole training history
            train_epoch_loss = torch.cat(
                (train_epoch_loss, loss.cpu().unsqueeze(0) / labels.cpu().size(0))
            )
            train_loss_log.append(loss.cpu().data / labels.cpu().size(0))

            # log accuracy for the current epoch and the whole training history
            pred_classes = torch.argmax(y_pred.cpu(), dim=-1)
            train_epoch_true_hits = torch.cat(
                (
                    train_epoch_true_hits,
                    (pred_classes == labels.cpu()).sum().unsqueeze(0),
                )
            )
            train_acc_log.append(
                (pred_classes == labels.cpu()).sum() / labels.cpu().shape[0]
            )

        # валидация
        val_epoch_loss, val_epoch_true_hits = torch.empty(0), torch.empty(0)
        model.eval()
        with torch.no_grad():
            for i, (imgs, labels) in enumerate(val_dataloader):
                if i * BATCHSIZE >= N_VALID_EXAMPLES:
                    break
                imgs, labels = imgs.to(device), labels.to(device)

                y_pred = model(imgs)
                loss = criterion(y_pred, labels)
                val_epoch_loss = torch.cat(
                    (val_epoch_loss, loss.cpu().unsqueeze(0) / labels.cpu().size(0))
                )

                pred_classes = torch.argmax(y_pred.cpu(), dim=-1)
                val_epoch_true_hits = torch.cat(
                    (
                        val_epoch_true_hits,
                        (pred_classes == labels.cpu()).sum().unsqueeze(0),
                    )
                )

        val_loss_log.append(val_epoch_loss.mean())
        val_acc_log.append(
            val_epoch_true_hits.sum()
            / val_epoch_true_hits.size(0)
            / val_dataloader.batch_size
        )
        accuracy = (val_epoch_true_hits.sum()
                / val_epoch_true_hits.size(0)
                / val_dataloader.batch_size).item()


        trial.report(accuracy, epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return accuracy


if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=75, timeout=1000)

    pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
    complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2024-01-08 17:18:58,448] A new study created in memory with name: no-name-5444bf66-0821-43c9-8c1e-0ad4693d18d9
[I 2024-01-08 17:22:36,315] Trial 0 finished with value: 0.6022727489471436 and parameters: {'n_layers': 2, 'optimizer': 'Adam', 'lr': 0.003298320467810311}. Best is trial 0 with value: 0.6022727489471436.
[I 2024-01-08 17:26:31,277] Trial 1 finished with value: 0.6335227489471436 and parameters: {'n_layers': 3, 'optimizer': 'Adam', 'lr': 0.03433531368025113}. Best is trial 1 with value: 0.6335227489471436.
[I 2024-01-08 17:30:32,165] Trial 2 finished with value: 0.6505681872367859 and parameters: {'n_layers': 3, 'optimizer': 'RMSprop', 'lr': 5.9100038868915736e-05}. Best is trial 2 with value: 0.6505681872367859.
[I 2024-01-08 17:34:34,115] Trial 3 finished with value: 0.4857954680919647 and parameters: {'n_layers': 3, 'optimizer': 'RMSprop', 'lr': 0.026466525591902382}. Best is trial 2 with value: 0.6505681872367859.
[I 2024-01-08 17:38:26,043] Trial 4 finished with value

Study statistics: 
  Number of finished trials:  5
  Number of pruned trials:  0
  Number of complete trials:  5
Best trial:
  Value:  0.6505681872367859
  Params: 
    n_layers: 3
    optimizer: RMSprop
    lr: 5.9100038868915736e-05


In [ ]:
import os

import optuna
from optuna.trial import TrialState
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
import torchvision
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

DEVICE = torch.device("cpu")
BATCHSIZE = 32
CLASSES = 4
DIR = os.getcwd()
EPOCHS = 10
N_TRAIN_EXAMPLES = BATCHSIZE * 45
N_VALID_EXAMPLES = BATCHSIZE * 11


def define_model(trial):
    # We optimize the number of layers, hidden units and dropout ratio in each layer.
    n_layers = trial.suggest_int("n_layers", 1, 4)
    layers = []
    in_channels = [3, 8, 16, 32]
    out_channels = [8, 16, 32, 64]
    dimension = [1, 8*111*111, 16*54*54, 32*26*26, 64*12*12]
    for i in range(n_layers):
        layers.append(nn.Conv2d(in_channels=in_channels[i], out_channels=out_channels[i], kernel_size=3))
        layers.append(nn.BatchNorm2d(out_channels[i]))
        layers.append(nn.ReLU())
        layers.append(nn.MaxPool2d(kernel_size=2))
    layers.append(nn.Flatten())
    layers.append(nn.Linear(dimension[n_layers], CLASSES))
    layers.append(nn.LogSoftmax(dim=1))
    return nn.Sequential(*layers)


def get_data():
    train_path = ('data_folder/Training/')
    test_path = ('data_folder/Testing/')
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    train_dataset = ImageFolder(root=train_path, transform=transform)
    test_dataset = ImageFolder(root=test_path, transform=transform)
    train_loader=DataLoader(
        train_dataset,
        batch_size=32, shuffle=True,
    )
    test_loader=DataLoader(
        test_dataset,
        batch_size=32, shuffle=True,
    )
    return train_loader, test_loader


def objective(trial):
    # Generate the model.
    model = define_model(trial).to(DEVICE)
    # Generate the optimizers.
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    # Get the FashionMNIST dataset.
    train_dataloader, val_dataloader = get_data()
    train_loss_log, train_acc_log, val_loss_log, val_acc_log = [], [], [], []
    # Training of the model.
    for epoch in range(EPOCHS):
        # тренировка
        train_epoch_loss, train_epoch_true_hits = torch.empty(0), torch.empty(0)
        model.train()
        for i, (imgs, labels) in enumerate(train_dataloader):
            # Limiting training data for faster epochs.
            if i * BATCHSIZE >= N_TRAIN_EXAMPLES:
                break
            imgs, labels = imgs.to(device), labels.to(device)

            y_pred = model(imgs)
            loss = criterion(y_pred, labels)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            # log loss for the current epoch and the whole training history
            train_epoch_loss = torch.cat(
                (train_epoch_loss, loss.cpu().unsqueeze(0) / labels.cpu().size(0))
            )
            train_loss_log.append(loss.cpu().data / labels.cpu().size(0))

            # log accuracy for the current epoch and the whole training history
            pred_classes = torch.argmax(y_pred.cpu(), dim=-1)
            train_epoch_true_hits = torch.cat(
                (
                    train_epoch_true_hits,
                    (pred_classes == labels.cpu()).sum().unsqueeze(0),
                )
            )
            train_acc_log.append(
                (pred_classes == labels.cpu()).sum() / labels.cpu().shape[0]
            )

        # валидация
        val_epoch_loss, val_epoch_true_hits = torch.empty(0), torch.empty(0)
        model.eval()
        with torch.no_grad():
            for i, (imgs, labels) in enumerate(val_dataloader):
                if i * BATCHSIZE >= N_VALID_EXAMPLES:
                    break
                imgs, labels = imgs.to(device), labels.to(device)

                y_pred = model(imgs)
                loss = criterion(y_pred, labels)
                val_epoch_loss = torch.cat(
                    (val_epoch_loss, loss.cpu().unsqueeze(0) / labels.cpu().size(0))
                )

                pred_classes = torch.argmax(y_pred.cpu(), dim=-1)
                val_epoch_true_hits = torch.cat(
                    (
                        val_epoch_true_hits,
                        (pred_classes == labels.cpu()).sum().unsqueeze(0),
                    )
                )

        val_loss_log.append(val_epoch_loss.mean())
        val_acc_log.append(
            val_epoch_true_hits.sum()
            / val_epoch_true_hits.size(0)
            / val_dataloader.batch_size
        )
        accuracy = (val_epoch_true_hits.sum()
                / val_epoch_true_hits.size(0)
                / val_dataloader.batch_size).item()


        trial.report(accuracy, epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return accuracy


if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=10, timeout=2000)

    pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
    complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2024-01-08 18:23:03,366] A new study created in memory with name: no-name-03b62c43-3553-4561-a46f-99b6daf1e643
[I 2024-01-08 18:26:10,580] Trial 0 finished with value: 0.6136363744735718 and parameters: {'n_layers': 1, 'optimizer': 'Adam', 'lr': 0.0017587555934523921}. Best is trial 0 with value: 0.6136363744735718.
[I 2024-01-08 18:30:58,831] Trial 1 finished with value: 0.6136363744735718 and parameters: {'n_layers': 4, 'optimizer': 'Adam', 'lr': 0.01186920310504067}. Best is trial 0 with value: 0.6136363744735718.
[I 2024-01-08 18:34:53,816] Trial 2 finished with value: 0.5397727489471436 and parameters: {'n_layers': 3, 'optimizer': 'RMSprop', 'lr': 0.004244768833274677}. Best is trial 0 with value: 0.6136363744735718.
[I 2024-01-08 18:38:56,255] Trial 3 finished with value: 0.4119318127632141 and parameters: {'n_layers': 4, 'optimizer': 'Adam', 'lr': 1.2658563588912667e-05}. Best is trial 0 with value: 0.6136363744735718.
[I 2024-01-08 18:42:46,200] Trial 4 finished with value: 

Study statistics: 
  Number of finished trials:  10
  Number of pruned trials:  3
  Number of complete trials:  7
Best trial:
  Value:  0.6846590638160706
  Params: 
    n_layers: 1
    optimizer: Adam
    lr: 0.00011706032846641942


In [ ]:
import os

import optuna
from optuna.trial import TrialState
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
import torchvision
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

DEVICE = torch.device("cpu")
BATCHSIZE = 32
CLASSES = 4
DIR = os.getcwd()
EPOCHS = 10
N_TRAIN_EXAMPLES = BATCHSIZE * 45
N_VALID_EXAMPLES = BATCHSIZE * 11


def define_model(trial):
    # We optimize the number of layers, hidden units and dropout ratio in each layer.
    n_layers = trial.suggest_int("n_layers", 1, 4)
    layers = []
    in_channels = [3, 8, 16, 32]
    out_channels = [8, 16, 32, 64]
    dimension = [1, 8 * 111 * 111, 16 * 54 * 54, 32 * 26 * 26, 64 * 12 * 12]
    for i in range(n_layers):
        layers.append(nn.Conv2d(in_channels=in_channels[i], out_channels=out_channels[i], kernel_size=3))
        layers.append(nn.BatchNorm2d(out_channels[i]))
        layers.append(nn.ReLU())
        layers.append(nn.MaxPool2d(kernel_size=2))
    layers.append(nn.Flatten())
    layers.append(nn.Linear(dimension[n_layers], CLASSES))
    layers.append(nn.LogSoftmax(dim=1))
    return nn.Sequential(*layers)


def get_data():
    train_path = ('data_folder/Training/')
    test_path = ('data_folder/Testing/')
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    train_dataset = ImageFolder(root=train_path, transform=transform)
    test_dataset = ImageFolder(root=test_path, transform=transform)
    train_loader = DataLoader(
        train_dataset,
        batch_size=32, shuffle=True,
    )
    test_loader = DataLoader(
        test_dataset,
        batch_size=32, shuffle=True,
    )
    return train_loader, test_loader


def objective(trial):
    # Generate the model.
    model = define_model(trial).to(DEVICE)
    # Generate the optimizers.
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    # Get the FashionMNIST dataset.
    train_dataloader, val_dataloader = get_data()
    train_loss_log, train_acc_log, val_loss_log, val_acc_log = [], [], [], []
    # Training of the model.
    for epoch in range(EPOCHS):
        # тренировка
        train_epoch_loss, train_epoch_true_hits = torch.empty(0), torch.empty(0)
        model.train()
        for i, (imgs, labels) in enumerate(train_dataloader):
            # Limiting training data for faster epochs.
            if i * BATCHSIZE >= N_TRAIN_EXAMPLES:
                break
            imgs, labels = imgs.to(device), labels.to(device)

            y_pred = model(imgs)
            loss = criterion(y_pred, labels)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            # log loss for the current epoch and the whole training history
            train_epoch_loss = torch.cat(
                (train_epoch_loss, loss.cpu().unsqueeze(0) / labels.cpu().size(0))
            )
            train_loss_log.append(loss.cpu().data / labels.cpu().size(0))

            # log accuracy for the current epoch and the whole training history
            pred_classes = torch.argmax(y_pred.cpu(), dim=-1)
            train_epoch_true_hits = torch.cat(
                (
                    train_epoch_true_hits,
                    (pred_classes == labels.cpu()).sum().unsqueeze(0),
                )
            )
            train_acc_log.append(
                (pred_classes == labels.cpu()).sum() / labels.cpu().shape[0]
            )

        # валидация
        val_epoch_loss, val_epoch_true_hits = torch.empty(0), torch.empty(0)
        model.eval()
        with torch.no_grad():
            for i, (imgs, labels) in enumerate(val_dataloader):
                if i * BATCHSIZE >= N_VALID_EXAMPLES:
                    break
                imgs, labels = imgs.to(device), labels.to(device)

                y_pred = model(imgs)
                loss = criterion(y_pred, labels)
                val_epoch_loss = torch.cat(
                    (val_epoch_loss, loss.cpu().unsqueeze(0) / labels.cpu().size(0))
                )

                pred_classes = torch.argmax(y_pred.cpu(), dim=-1)
                val_epoch_true_hits = torch.cat(
                    (
                        val_epoch_true_hits,
                        (pred_classes == labels.cpu()).sum().unsqueeze(0),
                    )
                )

        val_loss_log.append(val_epoch_loss.mean())
        val_acc_log.append(
            val_epoch_true_hits.sum()
            / val_epoch_true_hits.size(0)
            / val_dataloader.batch_size
        )
        accuracy = (val_epoch_true_hits.sum()
                    / val_epoch_true_hits.size(0)
                    / val_dataloader.batch_size).item()

        trial.report(accuracy, epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return accuracy


if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=20, timeout=2000)

    pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
    complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2024-01-08 18:59:47,193] A new study created in memory with name: no-name-8022807d-237c-4adc-9022-1d5d672768b3
[I 2024-01-08 19:03:48,214] Trial 0 finished with value: 0.6818181872367859 and parameters: {'n_layers': 3, 'optimizer': 'Adam', 'lr': 7.938652330546161e-05}. Best is trial 0 with value: 0.6818181872367859.
[I 2024-01-08 19:06:36,016] Trial 1 finished with value: 0.2954545319080353 and parameters: {'n_layers': 1, 'optimizer': 'SGD', 'lr': 0.027824206659105427}. Best is trial 0 with value: 0.6818181872367859.
[I 2024-01-08 19:10:52,705] Trial 2 finished with value: 0.40340909361839294 and parameters: {'n_layers': 4, 'optimizer': 'SGD', 'lr': 0.0003108239833523926}. Best is trial 0 with value: 0.6818181872367859.
[I 2024-01-08 19:13:33,914] Trial 3 finished with value: 0.6931818127632141 and parameters: {'n_layers': 1, 'optimizer': 'RMSprop', 'lr': 5.4321047942608654e-05}. Best is trial 3 with value: 0.6931818127632141.
[I 2024-01-08 19:17:43,168] Trial 4 finished with value:

Study statistics: 
  Number of finished trials:  17
  Number of pruned trials:  10
  Number of complete trials:  7
Best trial:
  Value:  0.7357954382896423
  Params: 
    n_layers: 3
    optimizer: Adam
    lr: 0.0009974157877713538
